## 2일차 공부 기록 ##

In [ ]:
for i in range(n):
    j = i - 1 # 인덱스 하나 전  j->i
    while j >= 0:
        if A[j] < A[i]:
            break
        j -= 1
    B[i] = A[j] if j >= 0 else None

In [1]:
def f4(A, n):
    i = 0
    j = n - 1
    while i < j:
        if A[i] < A[j]:
            A[i], A[j] = A[j], A[i]
        i += 1
        j -= 1
    return A
A = [8,2,7,5,3,4,2]
print(f4(A, len(A)))

[8, 4, 7, 5, 3, 2, 2]


# 하나하나 코드 분석하는 것은 효율적이지 않음. 일단 큰 틀에서 회독 후 기출 문제 중심으로 디테일 신경쓰기 

In [ ]:
# -*- coding: utf-8 -*-
# 스택 기반 계산기 최종 버전

class Stack:
    """
    Python 리스트를 활용한 간단한 스택 구현.
    강의 자료 43페이지의 동적 배열 기반 스택 클래스를 참고하여 작성.
    """
    def __init__(self):
        """스택을 초기화합니다."""
        self.items = []

    def push(self, val):
        """스택에 값을 추가합니다."""
        self.items.append(val)

    def pop(self):
        """스택의 마지막 값을 꺼내고 반환합니다."""
        if self.isEmpty():
            # 이 오류는 유효하지 않은 수식을 처리하기 위해 메인 블록에서 감지됩니다.
            raise IndexError("pop from an empty stack")
        return self.items.pop()

    def top(self):
        """스택의 마지막 값을 삭제하지 않고 반환합니다."""
        if self.isEmpty():
            raise IndexError("top from an empty stack")
        return self.items[-1]

    def isEmpty(self):
        """스택이 비어있는지 확인합니다."""
        return not self.items

    def __len__(self):
        """스택의 크기를 반환합니다."""
        return len(self.items)

def get_token_list(expr: str) -> list:
    """
    중위 표기식 문자열을 토큰(피연산자, 연산자) 리스트로 변환합니다.
    강의 자료에는 포함되지 않았으나, 입력 문자열의 전처리를 위해 새롭게 작성되었습니다.
    """
    tokens = []
    i = 0
    while i < len(expr):
        char = expr[i]

        if char.isspace():
            i += 1
            continue
        
        # 피연산자 (여러 자리의 숫자 및 소수점 처리)
        if char.isdigit() or char == '.':
            num_str = ""
            while i < len(expr) and (expr[i].isdigit() or expr[i] == '.'):
                num_str += expr[i]
                i += 1
            # '3.1.4'와 같이 잘못된 숫자 형식을 확인합니다.
            if num_str.count('.') > 1:
                return None
            tokens.append(num_str)
        # 연산자 또는 괄호
        elif char in "+-*/^()":
            tokens.append(char)
            i += 1
        # 허용되지 않은 문자
        else:
            return None
    return tokens

def infix_to_postfix(token_list: list) -> list:
    """
    중위 표기식 토큰 리스트를 후위 표기식으로 변환합니다.
    강의 자료 47페이지의 'Pseudo 코드: infix → postfix' 알고리즘을 기반으로 작성되었습니다.
    """
    if token_list is None:
        return None
        
    op_stack = Stack()
    postfix_list = []
    # 연산자 우선순위 정의 ('^'는 지수 연산을 위해 추가됨).
    precedence = {'(': 0, '+': 1, '-': 1, '*': 2, '/': 2, '^': 3}

    for token in token_list:
        try:
            # 토큰이 피연산자인지 확인하기 위해 float 변환을 시도합니다.
            float(token)
            postfix_list.append(token)
        except ValueError:
            # 토큰이 연산자인 경우
            if token == '(':
                op_stack.push(token)
            elif token == ')':
                try:
                    top_token = op_stack.pop()
                    while top_token != '(':
                        postfix_list.append(top_token)
                        top_token = op_stack.pop()
                except IndexError: # 괄호 쌍이 맞지 않는 경우
                    return None
            else: # 일반 연산자 (+, -, *, /, ^)
                # 우결합(right-associative) 연산자인 '^' 처리 (과제 요구사항으로 추가된 기능)
                if token == '^':
                    while not op_stack.isEmpty() and precedence.get(op_stack.top(), 0) > precedence.get(token, 0):
                        postfix_list.append(op_stack.pop())
                # 좌결합(left-associative) 연산자 처리
                else:
                    while not op_stack.isEmpty() and precedence.get(op_stack.top(), 0) >= precedence.get(token, 0):
                        postfix_list.append(op_stack.pop())
                op_stack.push(token)

    # 스택에 남아있는 모든 연산자를 결과 리스트에 추가합니다.
    while not op_stack.isEmpty():
        op = op_stack.pop()
        if op == '(': # 괄호 쌍이 맞지 않는 경우
            return None
        postfix_list.append(op)

    return postfix_list

def compute_postfix(token_list: list):
    """
    후위 표기식 토큰 리스트를 계산하여 결과를 반환합니다.
    강의 자료 48페이지의 'postfix 수식을 실제로 계산해보기' 알고리즘을 기반으로 작성되었습니다.
    """
    if token_list is None:
        return "INVALID_EXPRESSION"
        
    operand_stack = Stack()
    
    for token in token_list:
        try:
            # 피연산자는 float으로 변환하여 스택에 추가합니다.
            operand_stack.push(float(token))
        except ValueError:
            # 토큰이 연산자인 경우, 피연산자 두 개를 꺼내 계산합니다.
            try:
                op2 = operand_stack.pop()
                op1 = operand_stack.pop()
            except IndexError: # 연산에 필요한 피연산자가 부족한 경우.
                return "INVALID_EXPRESSION"

            if token == '+':
                operand_stack.push(op1 + op2)
            elif token == '-':
                operand_stack.push(op1 - op2)
            elif token == '*':
                operand_stack.push(op1 * op2)
            elif token == '/':
                if op2 == 0:
                    return "ZERO_DIVISION_ERROR"
                operand_stack.push(op1 / op2)
            elif token == '^': # 지수 연산자 처리 (과제 요구사항으로 추가된 기능)
                operand_stack.push(op1 ** op2)

    # 모든 연산 후, 최종 결과는 스택에 남은 유일한 값이어야 합니다.
    if len(operand_stack) == 1:
        return operand_stack.pop()
    else:
        # 스택에 값이 여러 개 남은 경우(예: '3 4 + 5')는 수식 오류입니다.
        return "INVALID_EXPRESSION"

# 메인 실행 블록
if __name__ == "__main__":
    try:
        expr = input()
        
        # 1단계: 입력 문자열을 토큰 리스트로 변환합니다.
        tokens = get_token_list(expr)
        if tokens is None:
            raise ValueError("Invalid token")

        # 2단계: 중위 표기식 토큰 리스트를 후위 표기식으로 변환합니다.
        postfix = infix_to_postfix(tokens)
        if postfix is None:
            raise ValueError("Invalid expression format")

        # 3단계: 후위 표기식을 계산합니다.
        result = compute_postfix(postfix)
        
        # 결과 출력 형식에 맞춰 출력합니다.
        if isinstance(result, str): # 결과가 에러 메시지(문자열)인 경우
            print(result)
        else: # 결과가 숫자인 경우
            print(f"{result:.3f}")
            
    except (ValueError, IndexError, TypeError):
        # 모든 잠재적 오류를 감지하여 지정된 오류 메시지를 출력합니다.
        print("INVALID_EXPRESSION")



# To do List 
1. 구름 stack 까지 전부 풀기 
2. 자료구조 1회독 하기 
3. Day 3 에다가 구름에 있는 코드 분석하기( 우선순위 낮음 )